## Segmenting and Clustering Neighborhoods in Toronto 

Week 3 assignment 

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

## Problem 1A - Scraping the Notebook from Wikipedia

In [50]:
#import necessary libraries 

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests 

In [51]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [52]:
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify()) using this, we find the table in a distinguishable section under the class 'wikitalbe sortable'

In [53]:
table1 = soup.find('table',{'class':'wikitable sortable'})
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    #used to find the header section of each data
    if (index == 0):
        columns = section
    else:
        data.append(section)

### Processing the data into a Pandas Dataframe

In [54]:
toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Problem 1B - Processing the Data

In [55]:
#3.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,Post Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [57]:
#Combining neighbourhoods into one area
toronto_df["Neighborhood"] = toronto_df.groupby("Post Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

In [58]:
#Removing duplicates
toronto_df = toronto_df.drop_duplicates()
#toronto_df

In [59]:

#update index to be Postal Code 
if(toronto_df.index.name != 'Post Code'):
    toronto_df = toronto_df.set_index('Post Code')

#toronto_df

### Shape Output 

In [60]:
toronto_df.shape

(103, 2)

## Question 2 - Getting coordinates of neighbourhoods

In [61]:
#importing the data from the CSV file
import pandas as pd
data_df = pd.read_csv('http://cocl.us/Geospatial_data')
#data_df.head()

In [62]:
#update index to be Postal Code 

if(data_df.index.name != 'Postal Code'):
    data_df = data_df.set_index('Postal Code')
    
data_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [63]:
#combining the columns into a list sorted by postal code
toronto_df = toronto_df.join(data_df)
toronto_df.head(8)

,Borough,Neighborhood,Latitude,Longitude
Post Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188


## Question 3 - please see separate file